In [ ]:
#SNEHA -- START -- IMPORT CLEAN DATA FROM RONDA'S EXPORT

In [78]:
#Import Libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import linear_model as lm
import seaborn as sns
import numpy as np
import scipy.stats as stats
from pathlib import Path
import requests
import json
from api_key import api_key
import time
from pprint import pprint

In [79]:
# CLEANED DATA IMPORT FROM RONDA"S EXPOPRTED CSV
csv_path = Path('Coffee_exp.csv')
coffee_df = pd.read_csv(csv_path, encoding="utf8")
coffee_df.columns

Index(['Unnamed: 0', 'Species', 'Owner', 'Country_of_Origin', 'Farm.Name',
       'Altitude', 'Region', 'Number.of.Bags', 'Bag.Weight', 'Harvest.Year',
       'Grading.Date', 'Variety', 'Processing_Method', 'Aroma', 'Flavor',
       'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity', 'Clean.Cup',
       'Sweetness', 'Cupper.Points', 'CoffeeScore', 'Moisture',
       'Category.One.Defects', 'Quakers', 'Color', 'Category.Two.Defects',
       'unit_of_measurement', 'altitude_low_meters', 'altitude_high_meters',
       'altitude_mean_meters'],
      dtype='object')

In [80]:
coffee_df_new = coffee_df.rename(columns={'Total.Cup.Points': 'CoffeeScore',
                                            'Country.of.Origin': 'Country_of_Origin',
                                         'Processing.Method': 'Processing_Method'})

In [ ]:
#SNEHA -- END -- IMPORT CLEAN DATA FROM RONDA"S EXPORT

In [ ]:
#TAYLOR -- START -- QUESTION 1 : The distribution scores across metrics -Compare Arabica and Robusta (Maybe stacked bar?)

In [ ]:
#TAYLOR -- END -- QUESTION 1 : The distribution scores across metrics -Compare Arabica and Robusta (Maybe stacked bar?)

In [ ]:
#SNEHA -- START -- QUESTION 2 : This team likes Columbian coffee: H0: Distribution of "Good" Scores across metrics is not significant in Columbia vs the world

In [81]:
#Creating High/Average/Low values for each coffee metric score
def Score_categories(DataFrame, Score_Columns): 
    for i, m in enumerate(Score_Columns):
        percentiles =DataFrame[{m}].quantile([(1/3),(2/3)])

        conditions = [(DataFrame[{m}] <= (1/3)),(DataFrame[{m}] > (1/3)) & (DataFrame[{m}] <= (2/3)),(DataFrame[{m}] > (2/3))]
        values = ['Low', 'Average', 'High']
        DataFrame[f"{m}_Bucket"] = np.select(conditions, values)

metrics = ['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body',
'Balance', 'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points',
'CoffeeScore', 'Moisture', 'Category.One.Defects', 'Quakers', 'Category.Two.Defects','altitude_mean_meters','Number.of.Bags']
Score_categories(coffee_df_new, metrics)

/var/folders/kl/d4qmnbgd10q401rmyb0lldk40000gn/T/ipykernel_69491/1552380861.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  percentiles =DataFrame[{m}].quantile([(1/3),(2/3)])
/var/folders/kl/d4qmnbgd10q401rmyb0lldk40000gn/T/ipykernel_69491/1552380861.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  conditions = [(DataFrame[{m}] <= (1/3)),(DataFrame[{m}] > (1/3)) & (DataFrame[{m}] <= (2/3)),(DataFrame[{m}] > (2/3))]
/var/folders/kl/d4qmnbgd10q401rmyb0lldk40000gn/T/ipykernel_69491/1552380861.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  conditions = [(DataFrame[{m}] <= (1/3)),(DataFrame[{m}] > (1/3)) & (DataFrame[{m}] <= (2/3)),(DataFrame[{m}] > (2/3))]
/var/folders/kl/d4qmnbgd10q401rmyb0lldk40000gn/T/ipykernel_69491/1552380861.py:6: FutureWarning: Passing a set as an in

In [82]:
coffee_df_new.columns

Index(['Unnamed: 0', 'Species', 'Owner', 'Country_of_Origin', 'Farm.Name',
       'Altitude', 'Region', 'Number.of.Bags', 'Bag.Weight', 'Harvest.Year',
       'Grading.Date', 'Variety', 'Processing_Method', 'Aroma', 'Flavor',
       'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity', 'Clean.Cup',
       'Sweetness', 'Cupper.Points', 'CoffeeScore', 'Moisture',
       'Category.One.Defects', 'Quakers', 'Color', 'Category.Two.Defects',
       'unit_of_measurement', 'altitude_low_meters', 'altitude_high_meters',
       'altitude_mean_meters', 'Aroma_Bucket', 'Flavor_Bucket',
       'Aftertaste_Bucket', 'Acidity_Bucket', 'Body_Bucket', 'Balance_Bucket',
       'Uniformity_Bucket', 'Clean.Cup_Bucket', 'Sweetness_Bucket',
       'Cupper.Points_Bucket', 'CoffeeScore_Bucket', 'Moisture_Bucket',
       'Category.One.Defects_Bucket', 'Quakers_Bucket',
       'Category.Two.Defects_Bucket', 'altitude_mean_meters_Bucket',
       'Number.of.Bags_Bucket'],
      dtype='object')

In [126]:
#Test
coffee_columbia = coffee_df_new.loc[(coffee_df_new["Country_of_Origin"]=="Colombia") & (coffee_df_new["Aroma_Bucket"]=="High"),"Aroma_Bucket"].count()
coffee_columbia

173

In [171]:
#High score distribution for Colombia
metrics = ['Aroma_Bucket', 'Flavor_Bucket','Aftertaste_Bucket', 'Acidity_Bucket', 'Body_Bucket', 'Balance_Bucket','Uniformity_Bucket', 'Clean.Cup_Bucket', 'Sweetness_Bucket',\
           'Cupper.Points_Bucket', 'CoffeeScore_Bucket', 'Moisture_Bucket','Category.One.Defects_Bucket', 'Quakers_Bucket','Category.Two.Defects_Bucket', 'altitude_mean_meters_Bucket',\
           'Number.of.Bags_Bucket']
colombia_dist =[]

def score_counts(cols):
    for i,m in enumerate(cols):
        colombia_dist.append(coffee_df_new.loc[(coffee_df_new["Country_of_Origin"]=="Colombia") & (coffee_df_new[m]=="High"),m].count())
                                
score_counts(metrics)
colombia_dist

[173,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 0,
 34,
 10,
 123,
 150,
 172]

In [172]:
#High score distribution for rest of the world
metrics = ['Aroma_Bucket', 'Flavor_Bucket','Aftertaste_Bucket', 'Acidity_Bucket', 'Body_Bucket', 'Balance_Bucket','Uniformity_Bucket', 'Clean.Cup_Bucket', 'Sweetness_Bucket',\
           'Cupper.Points_Bucket', 'CoffeeScore_Bucket', 'Moisture_Bucket','Category.One.Defects_Bucket', 'Quakers_Bucket','Category.Two.Defects_Bucket', 'altitude_mean_meters_Bucket',\
           'Number.of.Bags_Bucket']
All_dist =[]

def score_counts(cols):
    for i,m in enumerate(cols):
        All_dist.append(coffee_df_new.loc[(coffee_df_new["Country_of_Origin"]!="Colombia") & (coffee_df_new[m]=="High"),m].count())                           
score_counts(metrics)

All_dist_norm=(All_dist/sum(All_dist))*sum(colombia_dist)
All_dist_round = [int(round(i,0)) for i in All_dist_norm]
All_dist_round

[173,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 173,
 0,
 24,
 12,
 128,
 152,
 173]

In [173]:
# With 17 rows, the degree of freedom is 17-1 = 16
# With a p-value of 0.05, the confidence level is 1.00-0.05 = 0.95.
critical_value = stats.chi2.ppf(q = 0.95, df = 3)
critical_value

7.814727903251179

In [175]:
stats.chisquare(colombia_dist, All_dist_round)

/Users/snehathomas/anaconda3/lib/python3.10/site-packages/scipy/stats/_stats_py.py:7412: RuntimeWarning: invalid value encountered in divide
  terms = (f_obs_float - f_exp)**2 / f_exp


Power_divergenceResult(statistic=nan, pvalue=nan)

In [1]:
#SNEHA -- END -- QUESTION 2 : This team likes Columbian coffee: H0: Distribution of "Good" Scores across metrics is not significant in Columbia vs the world

In [ ]:
#GANESH -- START -- QUESTION 3 : Procesing Method - Visualize with scatter - metrics Scores vs Processing Method

In [ ]:
#GANESH -- END -- QUESTION 3 : Procesing Method - Visualize with scatter - metrics Scores vs Processing Method

In [ ]:
#TAYLOR -- START -- QUESTION 4 : ANOVA - Flavor, Aroma, Body - Distribution impact by Processing methods

In [ ]:
#TAYLOR -- END -- QUESTION 4 : ANOVA - Flavor, Aroma, Body - Distribution impact by Processing methods

In [ ]:
#AARON -- START -- QUESTION 5 : Color coded map for Species? Overall Scores?

In [ ]:
countries = ["Spain"]#, "England", "Ireland"]
url = f"https://api.geoapify.com/v1/geocode/search?"
country_coordinates = []
def color_coffee_map(countries):
    
     #coffee_df_new["Country"]
    for i, country in enumerate(countries):

        
        #target_country = countries #coffee_df_new["Country"]
        params = {'country' : country,
                  'apiKey': api_key,
                  }
        response = requests.get(url, params=params).json()
        print(response.status_code)
        print(json.dumps(response, indent=4, sort_keys=True))
        
        latitude = response["features"][0]["properties"]["lat"]
        longitude = response["features"][0]["properties"]["lon"]

        country_coordinates.append({
            "country" : country,
            "lat" : latitude,
            "lon" : longitude
        })
        #else: pass

        country_coordinates_df = pd.DataFrame(country_coordinates)
        print(country_coordinates_df)

        #add lat and long columns into df

    map = country_coordinates_df.hvplot.points(
            "lon",
            "lat",
            geo = True,
            tiles = "OSM",
            
            #size = "CoffeeScore",
            
            color = "country",
            frame_width = 800,
            frame_height = 600,
            scale = 0.5

           )
    return map


color_coffee_map(countries)


In [ ]:
#AARON -- END -- QUESTION 5 : Color coded map for Species? Overall Scores?